# Global Charts

## Datasets
- [Global Economic Monitor (GEM)](https://databank.worldbank.org/reports.aspx?source=1179)

- [GDP per capita, PPP (constant 2021 international $)](https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.KD)

- [Real effective exchange rate index (2010 = 100)](https://data.worldbank.org/indicator/PX.REX.REER)

## Preamble

In [ ]:
import pandas as pd
import wbdata
import plotly.express as px
import plotly.graph_objects as go

import dateutil.parser

## CPI vs. GDP per Capita (PPP) with REER Bubble Size

In [ ]:
CPI_ID = "CPTOTSAXN"
PPP_ID = "NY.GDP.PCAP.PP.KD"
REER2010_ID = "PX.REX.REER"

In [ ]:
df_raw = wbdata.get_dataframe(
    indicators={
        CPI_ID: "CPI",
        PPP_ID: "PPP",
        REER2010_ID: "REER2010",
    },
    parse_dates=True,
)
df_raw

df = df_raw.reset_index()
df = df.dropna()
df = df[df["date"] > dateutil.parser.parse("2009")]
df["Year"] = df["date"].apply(lambda date: date.year)
df = df.rename({
    "country": "Country",
}, axis=1)
df = df.drop("date", axis=1)
df = df[["Year", "Country", "CPI", "PPP", "REER2010"]] # Reorder columns.
df = df.reset_index(drop=True)

countries_with_all_years = df.groupby('Country')['Year'].apply(lambda years: set(range(2010, 2024)).issubset(set(years)))
valid_countries = countries_with_all_years[countries_with_all_years].index

df = df[df['Country'].isin(valid_countries)]
df

In [ ]:
fig = px.scatter(
    df,
    x="CPI",
    y="PPP",
    size="REER2010",
    color="Country",
    hover_name="Country",
    hover_data=["Year"],
    animation_frame="Year",
    title="CPI vs. GDP per Capita (PPP) with REER Bubble Size",
    labels={"CPI": "Consumer Price Index (CPI)", "PPP": "GDP per Capita (PPP)", "REER2010": "Real Effective Exchange Rate (2010)"},
    range_x=[0, 400],
    range_y=[0, 150_000],
)

# Border for all points.
fig.update_traces(marker=dict(
    line=dict(width=1, color='black')
))

# Thicker border for Singapore.
fig.update_traces(
    selector=dict(name='Singapore'),
    marker=dict(line=dict(width=5, color='black'))
)

fig.show()